<a href="https://colab.research.google.com/github/Azimoj/WebScraping/blob/main/WebScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Obtenir les derniers articles du Monde (**)

Obtenir le titre et le chapeau (description) des derniers articles sur https://www.lemonde.fr/actualite-en-continu/ (*)

Enregistrer les images correspondantes (***, @)

In [4]:
import requests
from bs4 import BeautifulSoup

 Obtenir le titre et les chapeaux

La première étape est de demander la page au serveur en utilisant la fonction requests.get.

In [5]:
page= requests.get("https://www.lemonde.fr/actualite-en-continu/")

In [6]:
page.status_code

200

Noter le code de réponse : 200 = Success.

On peut afficher le contenu HTML de la page avec l'attribut content.

In [ ]:
page.content

In [ ]:
soup = BeautifulSoup(page.content)

print(soup.prettify()) # pour un meilleur affichage

On peut ensuite extraire le texte depuis les tags qui nous intéressent.

Pour connaître le noms des tags pertinents, on utilise l'inspecteur du navigateur. On va se concentrer sur la section id="river", qui contient les articles :

In [ ]:
content = soup.find('section', {'id':'river'})
content

In [12]:
# Syntaxe alternative : la méthode select

content2 = soup.select("html body section#river .teaser")

# Cette méthode utilise la syntaxe CSS pour rechercher un tag.
# Elle est parfois plus rapide à écrire. Les tags successifs sont
# separés par des espaces, les # désignent une class et les . un id.
# On peut obtenir le chemin CSS avec clic droit dans l'inspecteur.

On isole les titres (h3) et les chapeaux (p, class='teaser__desc') des articles :

In [17]:
titres = content.find_all('h3')
titres

[<h3 class="teaser__title">Bronchiolite : la distribution du Beyfortus interrompue dans les pharmacies de ville</h3>,
 <h3 class="teaser__title">Ilia Iachine, opposant russe : « La véritable Russie ne combat pas les Ukrainiens sur le front. La véritable Russie est en prison »</h3>,
 <h3 class="teaser__title">Affaire Benalla : peine confirmée en appel pour Alexandre Benalla après les « violences du 1ᵉʳ-Mai »</h3>,
 <h3 class="teaser__title">Paris 2024 : les Russes ne sont pas exclus des Jeux paralympiques</h3>,
 <h3 class="teaser__title">L’inflation en France reste à un niveau élevé, la consommation des ménages en baisse, rapporte l’Insee</h3>,
 <h3 class="teaser__title">Epic Games, société créatrice du jeu vidéo « Fortnite », annonce le licenciement de plus de 800 personnes</h3>,
 <h3 class="teaser__title">Les Etats-Unis vont-ils connaître un nouveau « shutdown » ? Ce qu’il faut savoir sur cette bataille budgétaire et ses conséquences</h3>,
 <h3 class="teaser__title">Plus de 50 morts e

In [16]:
chapeaux = content.find_all('p', {'class': 'teaser__desc'})
chapeaux

[<p class="teaser__desc">L’engouement pour le nouveau traitement est tel que les commandes passées par les officines tardent à arriver. La direction générale de la santé a recommandé, vendredi 29 septembre, aux médecins de ville de suspendre les prescriptions du dosage pour les bébés de plus de 5 kilos.</p>,
 <p class="teaser__desc">L’homme politique membre de l’opposition, condamné à huit ans et demi de colonie pénitentiaire pour avoir dénoncé l’agression russe en Ukraine, a envoyé un texte à l’occasion d’un forum organisé par l’association Russie-Libertés, à Paris, les 29 et 30 septembre. « Le Monde » en publie de larges extraits.</p>,
 <p class="teaser__desc">En première instance, l’ancien collaborateur de l’Elysée avait été condamné à trois ans de prison, dont un an ferme sous bracelet électronique.</p>,
 <p class="teaser__desc">Le comité international paralympique a voté contre l’exclusion vendredi du comité russe. Une décision qui marque un infléchissement de la fermeté dont a fa

In [18]:
titres[0].get_text()

'Bronchiolite\xa0: la distribution du Beyfortus interrompue dans les pharmacies de ville'

Le package unidecode permet de décoder les caractères unicode pour nettoyer le texte obtenu :

In [19]:
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.1 MB/s eta 0:00:00


In [20]:
from unidecode import unidecode

unidecode('Attentats de 2016 à Bruxelles\xa0: Salah Abdeslam et neuf\xa0autres inculpés renvoyés aux assises')

'Attentats de 2016 a Bruxelles : Salah Abdeslam et neuf autres inculpes renvoyes aux assises'

In [ ]:
# Nettoyage sous forme de boucle
titres_clean = []
for t in titres:
    titres_clean.append( unidecode(t.get_text()) )
titres_clean

In [23]:
# Nettoyage sous forme de list comprehension (mieux)
titres = [ unidecode(t.get_text()) for t in titres ]
titres

['Bronchiolite : la distribution du Beyfortus interrompue dans les pharmacies de ville',
 'Ilia Iachine, opposant russe : << La veritable Russie ne combat pas les Ukrainiens sur le front. La veritable Russie est en prison >>',
 'Affaire Benalla : peine confirmee en appel pour Alexandre Benalla apres les << violences du 1er-Mai >>',
 'Paris 2024 : les Russes ne sont pas exclus des Jeux paralympiques',
 "L'inflation en France reste a un niveau eleve, la consommation des menages en baisse, rapporte l'Insee",
 'Epic Games, societe creatrice du jeu video << Fortnite >>, annonce le licenciement de plus de 800 personnes',
 "Les Etats-Unis vont-ils connaitre un nouveau << shutdown >> ? Ce qu'il faut savoir sur cette bataille budgetaire et ses consequences",
 "Plus de 50 morts et plusieurs dizaines de blesses au Pakistan dans une explosion pres d'une mosquee",
 "Comment gerer les sans-abri toujours plus nombreux dans l'ouest des Etats-Unis ? Les elus sous pression pour trouver des solutions ",


In [24]:
chapeaux = [unidecode(t.get_text()) for t in chapeaux]
chapeaux

["L'engouement pour le nouveau traitement est tel que les commandes passees par les officines tardent a arriver. La direction generale de la sante a recommande, vendredi 29 septembre, aux medecins de ville de suspendre les prescriptions du dosage pour les bebes de plus de 5 kilos.",
 "L'homme politique membre de l'opposition, condamne a huit ans et demi de colonie penitentiaire pour avoir denonce l'agression russe en Ukraine, a envoye un texte a l'occasion d'un forum organise par l'association Russie-Libertes, a Paris, les 29 et 30 septembre. << Le Monde >> en publie de larges extraits.",
 "En premiere instance, l'ancien collaborateur de l'Elysee avait ete condamne a trois ans de prison, dont un an ferme sous bracelet electronique.",
 "Le comite international paralympique a vote contre l'exclusion vendredi du comite russe. Une decision qui marque un inflechissement de la fermete dont a fait preuve jusque-la l'IPC.",
 "Les prix a la consommation ont augmente de 4,9 % sur un an en septem

Pour terminer, on peut éventuellement stocker les résultats dans un dataframe :

In [25]:
import pandas as pd

In [26]:
# Création du df à partir d'un dictionnaire
dico = {'titre':titres, 'chapeau':chapeaux}
pd.DataFrame(dico)

,titre,chapeau
0,Bronchiolite : la distribution du Beyfortus in...,L'engouement pour le nouveau traitement est te...
1,"Ilia Iachine, opposant russe : << La veritable...","L'homme politique membre de l'opposition, cond..."
2,Affaire Benalla : peine confirmee en appel pou...,"En premiere instance, l'ancien collaborateur d..."
3,Paris 2024 : les Russes ne sont pas exclus des...,Le comite international paralympique a vote co...
4,"L'inflation en France reste a un niveau eleve,...","Les prix a la consommation ont augmente de 4,9..."
5,"Epic Games, societe creatrice du jeu video << ...",Epic Games a revele jeudi la preparation d'un ...
6,Les Etats-Unis vont-ils connaitre un nouveau <...,Les lois de depenses permettant de financer le...
7,Plus de 50 morts et plusieurs dizaines de bles...,"Le ministre de l'information du Balouchistan, ..."
8,Comment gerer les sans-abri toujours plus nomb...,La canicule a mis a jour le nombre croissant d...
9,Les punaises de lit : comment se propagent-ell...,Alors que la Mairie de Paris parle d'une << re...


In [27]:
# Création du df à partir d'une liste
df = pd.DataFrame([titres, chapeaux]).T
df.columns = ['titre', 'chapeau']
df

,titre,chapeau
0,Bronchiolite : la distribution du Beyfortus in...,L'engouement pour le nouveau traitement est te...
1,"Ilia Iachine, opposant russe : << La veritable...","L'homme politique membre de l'opposition, cond..."
2,Affaire Benalla : peine confirmee en appel pou...,"En premiere instance, l'ancien collaborateur d..."
3,Paris 2024 : les Russes ne sont pas exclus des...,Le comite international paralympique a vote co...
4,"L'inflation en France reste a un niveau eleve,...","Les prix a la consommation ont augmente de 4,9..."
5,"Epic Games, societe creatrice du jeu video << ...",Epic Games a revele jeudi la preparation d'un ...
6,Les Etats-Unis vont-ils connaitre un nouveau <...,Les lois de depenses permettant de financer le...
7,Plus de 50 morts et plusieurs dizaines de bles...,"Le ministre de l'information du Balouchistan, ..."
8,Comment gerer les sans-abri toujours plus nomb...,La canicule a mis a jour le nombre croissant d...
9,Les punaises de lit : comment se propagent-ell...,Alors que la Mairie de Paris parle d'une << re...


In [28]:
# Utilisation de zip
list(zip(titres, chapeaux))

[('Bronchiolite : la distribution du Beyfortus interrompue dans les pharmacies de ville',
  "L'engouement pour le nouveau traitement est tel que les commandes passees par les officines tardent a arriver. La direction generale de la sante a recommande, vendredi 29 septembre, aux medecins de ville de suspendre les prescriptions du dosage pour les bebes de plus de 5 kilos."),
 ('Ilia Iachine, opposant russe : << La veritable Russie ne combat pas les Ukrainiens sur le front. La veritable Russie est en prison >>',
  "L'homme politique membre de l'opposition, condamne a huit ans et demi de colonie penitentiaire pour avoir denonce l'agression russe en Ukraine, a envoye un texte a l'occasion d'un forum organise par l'association Russie-Libertes, a Paris, les 29 et 30 septembre. << Le Monde >> en publie de larges extraits."),
 ('Affaire Benalla : peine confirmee en appel pour Alexandre Benalla apres les << violences du 1er-Mai >>',
  "En premiere instance, l'ancien collaborateur de l'Elysee ava

Note : jusqu'à maintenant, on a raisonné en cherchant tous les titres, puis tous les chapeaux, puis en les rassemblant. On aurait pu raisonner autrement, avec une approche plus "bottom-up" : chercher le titre et le chapeau d'un seul article, puis utiliser une boucle pour appliquer les calculs à tous les articles.

In [29]:
# Approche "bottom-up" : boucle pour extraire les infos de chaque article

articles = []
for article in content:
    try:
        title = article.find('h3').get_text()
        title = unidecode(title)
        chapeau = article.find('p').get_text(strip=True)
        chapeau = unidecode(chapeau)
        articles.append([title, chapeau])
    except:
        continue

pd.DataFrame(articles, columns=['titres', 'chapeaux'])

,titres,chapeaux
0,Bronchiolite : la distribution du Beyfortus in...,L'engouement pour le nouveau traitement est te...
1,"Ilia Iachine, opposant russe : << La veritable...","L'homme politique membre de l'opposition, cond..."
2,Affaire Benalla : peine confirmee en appel pou...,"En premiere instance, l'ancien collaborateur d..."
3,Paris 2024 : les Russes ne sont pas exclus des...,Le comite international paralympique a vote co...
4,"L'inflation en France reste a un niveau eleve,...","Les prix a la consommation ont augmente de 4,9..."
5,"Epic Games, societe creatrice du jeu video << ...",Epic Games a revele jeudi la preparation d'un ...
6,Les Etats-Unis vont-ils connaitre un nouveau <...,Les lois de depenses permettant de financer le...
7,Plus de 50 morts et plusieurs dizaines de bles...,"Le ministre de l'information du Balouchistan, ..."
8,Comment gerer les sans-abri toujours plus nomb...,La canicule a mis a jour le nombre croissant d...
9,Les punaises de lit : comment se propagent-ell...,Alors que la Mairie de Paris parle d'une << re...


In [30]:

len(articles)

60


 Télécharger les images (***, @)

On commence par rechercher toutes les images de la section river :